In this notebook, I wish to try to use the bigram approach. I gather all the bigrams in the corpus in the all2Grams dict. For every bigram, I keep a track of the frequency of thier occurances. However, if the bigram has a question word in it, then I inflate the count by multiplying it by k=5. 
As a result, when I take the top n bigrams, the bigrams that usually determine the type of question such as [what, is] etc are included in the training vectors. 

In [28]:
from string import punctuation

In [29]:
Question_Words = {'what', 'where', 'when','how','why','did','do','does','have','has','am','is','are','can','could','may','would','will','should'
"didn't","doesn't","haven't","isn't","aren't","can't","couldn't","wouldn't","won't","shouldn't"}

In [3]:
all2grams=dict()       #dict to store 
Gram2sentences=[]      #all sentence
label1=[]              #primary labels
label2=[]              #secondary labels
lab1Count=1               
lab2Count=1
label1dict=dict()
label2dict=dict()

for line in open('training.data','r',encoding="ISO-8859-1"):
    line=[word.lower() for word in line.strip().split(' ')]
    labels=line[0].split(':')
    line=[word for word in line[1:] if word not in punctuation]
    if(len(labels)!=2 or ' ' in line[0]):
        print('Formatting asssumption wrong')
        break
    sentence2=[]
    for i in range(int(len(line)/2)):
        gram2=(line[i],line[i+1])
        sentence2.append(gram2)
        if(gram2 in all2grams):
            if(gram2[0] in Question_Words):
                all2grams[gram2]+=5
            else:
                all2grams[gram2]+=1
        else:
            if(gram2[0] in Question_Words):
                all2grams[gram2]=5
            else:
                all2grams[gram2]=1
    Gram2sentences.append(sentence2)         
    
    if(not(label1dict.get(labels[0],0))):
        label1dict[labels[0]]=lab1Count
        lab1Count+=1
    if(not(label2dict.get(labels[1],0))):
        label2dict[labels[1]]=lab2Count
        lab2Count+=1
    label1.append(label1dict[labels[0]])
    label2.append(label2dict[labels[1]])


In [9]:
popularTokens2gram=[x[0] for x in sorted(all2grams.items(),key=lambda x:x[1],reverse=True)[:500]]


In [11]:
def wordVec(sent):
    vec=[]
    for gram in popularTokens2gram:
        if gram in sent:
            vec.append(1)
        else:
            vec.append(0)
    return vec   

In [15]:
import numpy as np

Store the vector representations of all the sentences in the corpus

In [13]:
vectors2Gram=[wordVec(eachSentence) for eachSentence in Gram2sentences]

In [17]:
trainVectors=np.array(vectors2Gram[:int(0.6*len(vectors2Gram))])
trainLabels1=np.array(label1[:int(0.6*len(vectors2Gram))])
trainLabels2=np.array(label2[:int(0.6*len(vectors2Gram))])

In [18]:
validVectors=np.array(vectors2Gram[int(0.6*len(vectors2Gram)):int(0.8*len(vectors2Gram))])
validLabels1=np.array(label1[int(0.6*len(vectors2Gram)):int(0.8*len(vectors2Gram))])
validLabels2=np.array(label2[int(0.6*len(vectors2Gram)):int(0.8*len(vectors2Gram))])

In [21]:
testVectors=np.array(vectors2Gram[int(0.8*len(vectors2Gram)):])
testLabels1=np.array(label1[int(0.8*len(vectors2Gram)):])
testLabels2=np.array(label2[int(0.8*len(vectors2Gram)):])

In [23]:
from sklearn import svm

In [24]:
for c in [100,1000,10**5]:
    clf=svm.SVC(C=c, kernel='rbf', gamma='auto', tol=0.001, class_weight='balanced', max_iter=-1,\
                decision_function_shape='ovr')
    clf.fit(trainVectors,trainLabels1)
    preds=clf.predict(validVectors)
    accu=sum([1 if i==j else 0 for i,j in zip(preds,validLabels1)])/len(validLabels1)
    print(accu)

0.7488673139158576
0.7433656957928803
0.7498381877022654


Choosing c=10**5 based on the above results

In [25]:
clf=svm.SVC(C=10**5, kernel='rbf', gamma='auto', tol=0.001, class_weight='balanced', max_iter=-1,\
                decision_function_shape='ovr')
clf.fit(np.vstack([trainVectors,validVectors]),np.hstack([trainLabels1,validLabels1]))
preds=clf.predict(testVectors)
accu=sum([1 if i==j else 0 for i,j in zip(preds,testLabels1)])/len(testLabels1)
print('Test accuracy obtained for primary label is ',accu)

Test accuracy obtained for primary label is  0.72662568747978


In [26]:
for c in [100,1000,10**5]:
    clf=svm.SVC(C=c, kernel='rbf', gamma='auto', tol=0.001, class_weight='balanced', max_iter=-1,\
                decision_function_shape='ovr')
    clf.fit(trainVectors,trainLabels2)
    preds=clf.predict(validVectors)
    accu=sum([1 if i==j else 0 for i,j in zip(preds,validLabels2)])/len(validLabels2)
    print(accu)

0.5420711974110033
0.6061488673139158
0.6087378640776699


In [27]:
clf=svm.SVC(C=10**5, kernel='rbf', gamma='auto', tol=0.001, class_weight='balanced', max_iter=-1,\
                decision_function_shape='ovr')
clf.fit(np.vstack([trainVectors,validVectors]),np.hstack([trainLabels2,validLabels2]))
preds=clf.predict(testVectors)
accu=sum([1 if i==j else 0 for i,j in zip(preds,testLabels2)])/len(testLabels2)
print("Test accuracy obtained for secondary label is",accu)

Test accuracy obtained for secondary label is 0.5454545454545454


I notice that this model performs worse than the unigram model. As I glanced through the dataset, the reason became clear. The bigrams are not repeating enough. A few bigrams repeat very frequently, but most bigrams have a very low count. 